In [ ]:
"""
This is a sample algorithm on Quantopian using the order_optimal_portfolio.

"""

# Import necessary Pipeline modules
from quantopian.pipeline import Pipeline  
from quantopian.pipeline.data import EquityPricing, factset  
from quantopian.pipeline.domain import US_EQUITIES  
from quantopian.research import run_pipeline
import quantopian.pipeline.factors as Factors
import quantopian.pipeline.filters as Filters
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.data import Fundamentals  
from quantopian.pipeline.factors.fundamentals import MarketCap
import numpy as np
import pandas as pd
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.data import morningstar as mstar
from quantopian.pipeline.data.factset.estimates import PeriodicConsensus
from quantopian.pipeline import Pipeline
from zipline.api import attach_pipeline, pipeline_output

# Import specific filters and factors which will be used
from quantopian.pipeline.filters import QTradableStocksUS, Q500US, StaticAssets
from quantopian.pipeline.factors import SimpleMovingAverage, Returns

# Import datasets which will be used
from quantopian.pipeline.data.sentdex import sentiment

# import optimize
import quantopian.optimize as opt
 
# Import pandas and numpy
import pandas as pd
import numpy as np

def make_pipeline():
    """
    Define a pipeline.
    
    We break this code out into its own function to make it easier to
    test and modify in isolation. In particular, this function can be
    copy/pasted into research and run by itself.
    Parameters
    """
    my_universe = Q500US()
    
    # Create any needed factors.
    # Define a 5-day simple moving average sentiment factor.
    mean_sentiment_5day = SimpleMovingAverage(inputs=[sentiment.sentiment_signal],
                                              window_length=5,
                                              mask=my_universe)
    
    returns = Returns(window_length = 20, mask=my_universe)
    
    # Create any filters based upon these factors
    biggest_gainers = returns.top(100)
    smallest_gainers = returns.bottom(100)
    
    latest_close = EquityPricing.close.latest
    market_cap =  Fundamentals.market_cap.latest

      
    # Create our pipeline
    pipe = Pipeline(
        columns={
            'price': latest_close,
            'mean_sentiment_5day': mean_sentiment_5day,
            'biggest_gainers': biggest_gainers,
            'smallest_gainers': smallest_gainers,
            'market_cap': market_cap
        },
        screen=my_universe
    )
    
    return pipe  
    
def place_orders_using_optimize(context, data):
    """
    Use Optimize to place orders all at once
    """
    # Fetch yesterdays data
    data = pipeline_output('my_pipe')
    
    # Select the sentiment for a single asset using the 'at' method
    # Record the value to see what's going on
    apple = symbol('AAPL')
    apple_sentiment = data.at[apple, 'mean_sentiment_5day']
    record(sentiment=apple_sentiment, state=np.sign(apple_sentiment))
    
    # Logic based upon apple sentiment
    if apple_sentiment > 0:
        # positive sentiment. Open the high gainers with a equal positive (long) weight.
        open_these = data.query('biggest_gainers').index
        weight = 1.0 / len(open_these)
    else:
        # negative sentiment. Open the smallest gainers with equal negative (short) weight.
        open_these = data.query('smallest_gainers').index
        weight = -1.0 / len(open_these)        

    # Make a series of the securities and associated weights
    weights = pd.Series(index=open_these, data=weight)
    
    # Execute the order_optimal_portfolio method with TargetWeights objective
    order_optimal_portfolio(
        objective = opt.TargetWeights(weights),
        constraints = []
    )

In [ ]:
my_pipe = make_pipeline()

result = run_pipeline(my_pipe, '2020-06-15','2020-06-15')
result.index.rename(['date', 'stock'], inplace=True)  
oneday = result.loc[('2020-06-15')]

oneday.head()

In [ ]:
oneday = oneday.sort_values(by=['market_cap'], ascending=False)

In [ ]:
oneday[oneday['mean_sentiment_5day'].isnull()]

In [ ]:
google = symbols('UBER').sid
google_sentiment = sentiment[ (sentiment.sid==google) ]
google_sentiment.peek()

In [ ]:
google_sentiment.peek()